In [1]:
# RSI PowerZone Weekly
import indicator
import numpy as np
import pandas as pd
import price
import trade_statistics
from bokeh.io import curdoc
from bokeh.models import HoverTool, NumeralTickFormatter
from bokeh.plotting import figure, show

In [2]:
# Trade parameters.
exchange = "LSE"
tidm = "DOTD"
filename = f"{exchange}_{tidm}_prices.csv"
history_start = "2003-03-19"
history_end = "2021-09-24"
p_sma = 40  # Simple Moving Average (SMA) look back period.
p_rsi = 4  # Relative Strength Index (RSI) look back period.
pz1 = 30  # First RSI PowerZone entry level.
pz2 = 25  # Second RSI PowerZone entry level.
pze = 75  # RSI PowerZone exit level.
position_size = 10000  # Position size in major currency unit.
charges = 6  # Commission per trade (round trip).
stamp_duty = 0.0  # Stamp Duty percentage.
stop_loss = 20  # Stop Loss percentage.

In [3]:
def state_signal(entry_signal, exit_signal, period):
    """Calculate trade state signals."""
    df = pd.concat([entry_signal, exit_signal], axis=1)
    df.columns = ["entry", "exit"]
    df["state"] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], "entry"] == 1 and df.loc[df.index[i - 1], "state"] == 0:
            df.loc[df.index[i], "state"] = 1
        elif df.loc[df.index[i], "exit"] == 1:
            df.loc[df.index[i], "state"] = 0
        else:
            df.loc[df.index[i], "state"] = df.loc[df.index[i - 1], "state"]
    return df.state


def trade_list(close, entry_flag, exit_flag):
    """Generate trade list."""
    ref = pd.concat([close, entry_flag, exit_flag], axis=1)
    ref.columns = ["close", "entry_flag", "exit_flag"]
    df = pd.DataFrame(columns=["entry_price", "exit_date", "exit_price"])
    df.entry_price = ref.close[ref.entry_flag == 1]
    df.exit_date = ref.index[ref.exit_flag == 1]
    df.exit_price = ref.close[ref.exit_flag == 1].values
    return df

In [4]:
# Import weekly closing prices.
df = price.weekly_close(exchange, tidm, history_start, history_end)

In [5]:
# Simple moving average (SMA).
df["sma"] = df.close.rolling(p_sma).mean()
sma = df.sma

# Relative Strength Index (RSI).
df["rsi"] = indicator.rsi(df.close, p_rsi)

In [6]:
# Check closing price is above SMA.
df["trend"] = np.where(df.close > df.sma, 1, 0)

# Check for RSI cross below first PowerZone.
df["pz1"] = np.where(np.logical_and(df.rsi.shift(periods=1) >= pz1, df.rsi < pz1), 1, 0)

# Check whether RSI is below second PowerZone and price is above SMA.
df["pz2"] = np.where(np.logical_and(df.trend == 1, df.rsi < pz2), 1, 0)

# Check for RSI cross above exit level.
df["pze"] = np.where(np.logical_and(df.rsi.shift(periods=1) < pze, df.rsi >= pze), 1, 0)

# Trade #1 entry signal.
df["sig1"] = np.where(np.logical_and(df.trend == 1, df.pz1 == 1), 1, 0)

# Trade #1 state signal.
df["ss1"] = state_signal(df.sig1, df.pze, p_sma)

# Trade #2 entry signal.
df["sig2"] = np.where(np.logical_and(df.ss1.shift(periods=1) == 1, df.pz2 == 1), 1, 0)

# Trade #2 state signal.
df["ss2"] = state_signal(df.sig2, df.pze, p_sma)

# Trade entry & exit flags.
df["en1"] = np.where(np.logical_and(df.ss1 == 1, df.ss1.shift(periods=1) == 0), 1, 0)
df["ex1"] = np.where(np.logical_and(df.ss1 == 0, df.ss1.shift(periods=1) == 1), 1, 0)
df["en2"] = np.where(np.logical_and(df.ss2 == 1, df.ss2.shift(periods=1) == 0), 1, 0)
df["ex2"] = np.where(np.logical_and(df.ss2 == 0, df.ss2.shift(periods=1) == 1), 1, 0)

In [7]:
# Trade list based on RSI exit only (no Stop Loss).

# Trade List #1.
td1 = trade_list(df.close, df.en1, df.ex1)

# Trade List #2.
td2 = trade_list(df.close, df.en2, df.ex2)

# Create full trade list.
td = pd.concat([td1, td2])

# Sort trades in ascending date order.
td = td.sort_index()

# Append trade percentage price change to trade list.
td["chg_pct"] = ((td.exit_price - td.entry_price) / td.entry_price) * 100

In [8]:
# Trade list incorporating Stop Loss.
tdm = pd.DataFrame(columns=["entry_price", "exit_date", "exit_price"])
tdm.entry_price = td.entry_price
tdm.exit_date = td.exit_date
tdm.exit_price = td.exit_price

for i in range(0, len(td)):
    # Loop through trade list based on RSI exit and check if stop is triggered.
    ps = pd.DataFrame(columns=["close", "stop", "exit"])
    ps.close = df.close.loc[td.index[i] : td.exit_date[i]]
    ps.stop = ps.close[0] * (1 - (stop_loss / 100))
    ps.exit = np.where(ps.close < ps.stop, 1, 0)

    # Create modified trade list incorporating Stop Loss.
    if ps.exit.sum() > 0:
        exit_date = ps.exit.idxmax(axis=1, skipna=True)
        tdm.loc[tdm.index[i], "exit_date"] = exit_date
        tdm.loc[tdm.index[i], "exit_price"] = ps.close.loc[exit_date]
    else:
        tdm.loc[tdm.index[i], "exit_date"] = td.loc[td.index[i], "exit_date"]
        tdm.loc[tdm.index[i], "exit_price"] = td.loc[td.index[i], "exit_price"]

In [9]:
# Cumulative profit after costs.
tdm["weeks"] = (tdm.exit_date - tdm.index) / np.timedelta64(1, "W")
tdm["chg_pct"] = ((tdm.exit_price - tdm.entry_price) / tdm.entry_price) * 100
tdm["charges"] = charges
tdm["stamp_duty"] = position_size * (stamp_duty / 100)
tdm["profit"] = position_size * (tdm.chg_pct / 100) - tdm.charges - tdm.stamp_duty
tdm["cum_profit"] = tdm.profit.cumsum()
cum_profit = tdm.cum_profit
tdm.round(2)

,entry_price,exit_date,exit_price,weeks,chg_pct,charges,stamp_duty,profit,cum_profit
date,,,,,,,,,
2012-03-23,0.10,2012-07-06,0.12,15.0,20.99,6,0.0,2092.77,2092.77
2012-03-30,0.10,2012-07-06,0.12,14.0,25.64,6,0.0,2558.10,4650.87
2012-08-24,0.11,2012-09-21,0.12,4.0,11.11,6,0.0,1105.11,5755.98
2012-08-31,0.11,2012-09-21,0.12,3.0,11.11,6,0.0,1105.11,6861.09
2013-05-31,0.15,2013-07-19,0.15,7.0,3.36,6,0.0,330.13,7191.22
2013-06-07,0.15,2013-07-19,0.15,6.0,3.36,6,0.0,330.13,7521.36
2014-05-23,0.32,2014-07-25,0.34,9.0,7.51,6,0.0,744.99,8266.35
2014-08-15,0.32,2014-10-03,0.34,7.0,6.20,6,0.0,614.16,8880.50
2014-08-22,0.32,2014-10-03,0.34,6.0,6.20,6,0.0,614.16,9494.66


In [10]:
# Trade Statistics
ts = trade_statistics.summary(tidm, sma, position_size, tdm)
ts

TIDM                              DOTD
Start Date                  2011-12-30
End Date                    2021-09-24
Analysis Years                     9.7
Position Size                10,000.00
Net Profit                   14,263.97
Annual %                           9.5
Charges                          84.00
Stamp Duty                        0.00
Total Trades                        14
Winning Trades                      14
Losing Trades                        0
Winning %                        100.0
Trades per Year                    1.4
Average Profit %                  10.2
Average Profit                1,018.86
Average Weeks                      8.5
Average Winning Profit %          10.2
Average Winning Profit        1,018.86
Average Winning Weeks              8.5
Average Losing Profit %            nan
Average Losing Profit              nan
Average Losing Weeks               nan
dtype: object

In [ ]:
# Plot Profit vs Date.
x = tdm.index
y = tdm.cum_profit

curdoc().theme = "dark_minimal"

p = figure(
    title=tidm,
    sizing_mode="stretch_width",
    tools=[HoverTool()],
    tooltips="Data point @x has the value @y{0,0.00}",
    x_axis_type="datetime",
    x_axis_label="Date",
    y_axis_label="Profit",
)

p.border_fill_color = (28, 28, 28)
p.xgrid.grid_line_color = (119, 119, 119)
p.ygrid.grid_line_color = (119, 119, 119)
p.yaxis[0].formatter = NumeralTickFormatter(format="0.00")

p.line(x, y, line_color=(92, 230, 174), line_width=2)
show(p)